<a href="https://colab.research.google.com/github/cscholles/SEO/blob/master/Categorize_search_intention_in_bulk_using_semrush_and_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries and Data

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import re #regex

In [2]:
#google mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#substitute path
df_kwstudy=pd.read_csv('/content/drive/My Drive/Colab Notebooks/carolinescholles.com-organic.Positions-br-20200811-2020-08-12T17_47_30Z.csv') #path to csv file

In [4]:
df_kwstudy.head(3) #let's see what we've just imported

,Keyword,Position,Previous position,Search Volume,Keyword Difficulty,CPC,URL,Traffic,Traffic (%),Traffic Cost,Competition,Number of Results,Trends,Timestamp,SERP Features by Keyword
0,nuvem de palavras,89,0,9900,69.71,25.96,https://carolinescholles.com/nuvem-de-palavras...,0,0,0.0,0.03,11200000,"[45,45,45,55,82,67,100,82,67,55,82,100]",2020-08-11,"Knowledge panel, Site links, Reviews, Video Ca..."
1,nuvem de palavras online,85,85,1600,70.70,6.80,https://carolinescholles.com/nuvem-de-palavras...,0,0,0.0,0.05,34100000,"[25,42,42,54,67,79,100,100,67,54,79,79]",2020-08-09,"Knowledge panel, Site links, Reviews, Video Ca..."
2,criar nuvem de palavras,40,40,480,67.08,6.49,https://carolinescholles.com/nuvem-de-palavras...,0,0,2.0,0.07,12300000,"[54,67,54,54,82,82,100,82,67,54,67,82]",2020-08-05,"Image pack, Reviews, Video Carousel, People al..."


In [5]:
df_kwstudy.drop(['Previous position','CPC','Traffic','Traffic (%)','Traffic Cost', 'Competition', 'Number of Results', 'Trends', 'Timestamp'], axis=1, inplace=True)


In [6]:
df_kwstudy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Keyword                   13 non-null     object 
 1   Position                  13 non-null     int64  
 2   Search Volume             13 non-null     int64  
 3   Keyword Difficulty        13 non-null     float64
 4   URL                       13 non-null     object 
 5   SERP Features by Keyword  13 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 752.0+ bytes


In [7]:
df_kwstudy.head(3) #let's see our new dataframe

,Keyword,Position,Search Volume,Keyword Difficulty,URL,SERP Features by Keyword
0,nuvem de palavras,89,9900,69.71,https://carolinescholles.com/nuvem-de-palavras...,"Knowledge panel, Site links, Reviews, Video Ca..."
1,nuvem de palavras online,85,1600,70.70,https://carolinescholles.com/nuvem-de-palavras...,"Knowledge panel, Site links, Reviews, Video Ca..."
2,criar nuvem de palavras,40,480,67.08,https://carolinescholles.com/nuvem-de-palavras...,"Image pack, Reviews, Video Carousel, People al..."


#Search Intent Categorization

In [9]:
df_kwstudy['SERP Features'] = df_kwstudy['SERP Features by Keyword'].astype(str)
SERP_Features = df_kwstudy['SERP Features']

In [10]:
def word_count(SERP_Features, word):
      cont = 0
      SERP_Features = SERP_Features.lower()
      if word in SERP_Features.lower():
          return SERP_Features.count(word)
      else:
          return 0

In [11]:
df_kwstudy['Knowledge Panel'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'panel'))/10
df_kwstudy['Top Stories'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'stories'))/10
df_kwstudy['Twitter'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'twitter'))/10
df_kwstudy['Featured snippet'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'snippet'))/10
df_kwstudy['People Also Ask'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'also'))/10
df_kwstudy['FAQ'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'faq'))/10
df_kwstudy['Image'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'image'))/10
df_kwstudy['Instant Answer'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'instant'))/10
df_kwstudy['Site Links'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'links'))/10
df_kwstudy['Local pack'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'local'))/10
df_kwstudy['Video'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'video'))/10
df_kwstudy['Reviews'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'reviews'))/10
df_kwstudy['AMP'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'AMP'))/10
df_kwstudy['Adwords'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'adwords'))/10
df_kwstudy['Shopping Ads'] = df_kwstudy['SERP Features'].apply(lambda x: word_count(x, 'shopping'))/10

In [12]:
df_kwstudy['SERP Features Degree']=df_kwstudy['Knowledge Panel'] + df_kwstudy['Top Stories'] + df_kwstudy['Twitter'] + df_kwstudy['Featured snippet'] + df_kwstudy['People Also Ask'] + df_kwstudy['Image'] + df_kwstudy['Instant Answer'] + df_kwstudy['Site Links'] + df_kwstudy['Local pack'] + df_kwstudy['Video'] + df_kwstudy['Reviews'] + df_kwstudy['AMP'] + df_kwstudy['Adwords'] + df_kwstudy['Shopping Ads']


In [13]:
#percentual por intenção
df_kwstudy['Know'] = df_kwstudy['Knowledge Panel'] + df_kwstudy['Top Stories'] + df_kwstudy['Twitter'] + df_kwstudy['Featured snippet'] + df_kwstudy['People Also Ask'] + df_kwstudy['Image'] + df_kwstudy['Instant Answer'] + df_kwstudy['Site Links'] + df_kwstudy['Video']
df_kwstudy['Do'] = df_kwstudy['Adwords'] + df_kwstudy['Shopping Ads'] + df_kwstudy['Reviews']
df_kwstudy['Local'] = df_kwstudy['Local pack']

In [14]:
df_kwstudy['Predominant Intention'] = df_kwstudy[['Know','Do','Local']].idxmax(axis=1)


In [15]:
def classifier(row):
  if row["Adwords"] > 0 or row["Shopping Ads"] > 0:
      return "Do"
  elif row["Reviews"] > 0:
      return "Evaluate | Do"
  elif row["Local pack"] > 0:
      return "Local"
  elif row["Top Stories"] > 0 or row["Twitter"] > 0:
      return "News"
  else:
      return "Not Identified"

In [16]:
df_kwstudy["Secondary Intention"] = df_kwstudy.apply(classifier, axis=1)

In [17]:
df_kwstudy.head(3)

,Keyword,Position,Search Volume,Keyword Difficulty,URL,SERP Features by Keyword,SERP Features,Knowledge Panel,Top Stories,Twitter,Featured snippet,People Also Ask,FAQ,Image,Instant Answer,Site Links,Local pack,Video,Reviews,AMP,Adwords,Shopping Ads,SERP Features Degree,Know,Do,Local,Predominant Intention,Secondary Intention
0,nuvem de palavras,89,9900,69.71,https://carolinescholles.com/nuvem-de-palavras...,"Knowledge panel, Site links, Reviews, Video Ca...","Knowledge panel, Site links, Reviews, Video Ca...",0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.5,0.4,0.1,0.0,Know,Evaluate | Do
1,nuvem de palavras online,85,1600,70.70,https://carolinescholles.com/nuvem-de-palavras...,"Knowledge panel, Site links, Reviews, Video Ca...","Knowledge panel, Site links, Reviews, Video Ca...",0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.5,0.4,0.1,0.0,Know,Evaluate | Do
2,criar nuvem de palavras,40,480,67.08,https://carolinescholles.com/nuvem-de-palavras...,"Image pack, Reviews, Video Carousel, People al...","Image pack, Reviews, Video Carousel, People al...",0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.4,0.3,0.1,0.0,Know,Evaluate | Do


In [18]:
df_kwstudy.drop(['SERP Features'], axis=1, inplace=True)

In [19]:
df_kwstudy.head(3)

,Keyword,Position,Search Volume,Keyword Difficulty,URL,SERP Features by Keyword,Knowledge Panel,Top Stories,Twitter,Featured snippet,People Also Ask,FAQ,Image,Instant Answer,Site Links,Local pack,Video,Reviews,AMP,Adwords,Shopping Ads,SERP Features Degree,Know,Do,Local,Predominant Intention,Secondary Intention
0,nuvem de palavras,89,9900,69.71,https://carolinescholles.com/nuvem-de-palavras...,"Knowledge panel, Site links, Reviews, Video Ca...",0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.5,0.4,0.1,0.0,Know,Evaluate | Do
1,nuvem de palavras online,85,1600,70.70,https://carolinescholles.com/nuvem-de-palavras...,"Knowledge panel, Site links, Reviews, Video Ca...",0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.5,0.4,0.1,0.0,Know,Evaluate | Do
2,criar nuvem de palavras,40,480,67.08,https://carolinescholles.com/nuvem-de-palavras...,"Image pack, Reviews, Video Carousel, People al...",0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.4,0.3,0.1,0.0,Know,Evaluate | Do


In [ ]:
df_kwstudy.to_csv('kwfinal.csv')